# Azure Environment Variables

In [1]:
%env AZURE_SUBSCRIPTION_ID=3c894f7d-87fb-4d7c-957c-b17c5fd4c6b9

# %env AZURE_STORAGE_CONNECTION_STRING=DefaultEndpointsProtocol=https;AccountName=mediastorkiw83wi;AccountKey=IJL5yUcYx99waESEOlZkJmlEhitknAjHp0SnrFaXOqz6HF68aATNMR5NaKWl7+TBQHRiPJtc8WAr8jcW4THEyQ==;EndpointSuffix=core.windows.net


env: AZURE_SUBSCRIPTION_ID=3c894f7d-87fb-4d7c-957c-b17c5fd4c6b9


In [2]:
import os, random

In [3]:
# Import the needed management objects from the libraries. The azure.common library
# is installed automatically with the other libraries.
from azure.identity import AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.storage import StorageManagementClient


# Random postfix for resources

In [4]:
#random.seed(0)
postfix = random.randint(1,100000)

# Credentials and Resource Client

In [5]:
# Acquire a credential object using CLI-based authentication.
credential = AzureCliCredential()

# Retrieve subscription ID from environment variable.
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]

# Obtain the management object for resources.
resource_client = ResourceManagementClient(credential, subscription_id)

# Create resource group

In [6]:
# Constants we need in multiple places: the resource group name and the region
# in which we provision resources. You can change these values however you want.
RESOURCE_GROUP_NAME = f"rg-az204-storage-{postfix:05}"
LOCATION = "westeurope"

# Step 1: Provision the resource group.

rg_result = resource_client.resource_groups.create_or_update(RESOURCE_GROUP_NAME, { "location": LOCATION })

print(f"Provisioned resource group {rg_result.name}")

# For details on the previous code, see Example: Provision a resource group
# at https://docs.microsoft.com/azure/developer/python/azure-sdk-example-resource-group

Provisioned resource group rg-az204-storage-28344


# Delete resource group

In [7]:
# Optional lines to delete the resource group. begin_delete is asynchronous.
# poller = resource_client.resource_groups.begin_delete(rg_result.name)
# result = poller.result()

# Create storage account

In [8]:
# Step 2: Provision the storage account, starting with a management object.

storage_client = StorageManagementClient(credential, subscription_id)

# This example uses the CLI profile credentials because we assume the script
# is being used to provision the resource in the same way the Azure CLI would be used.

STORAGE_ACCOUNT_NAME = f"storage{postfix:05}"

# You can replace the storage account here with any unique name. A random number is used
# by default, but note that the name changes every time you run this script.
# The name must be 3-24 lower case letters and numbers only.


# Check if the account name is available. Storage account names must be unique across
# Azure because they're used in URLs.
availability_result = storage_client.storage_accounts.check_name_availability(
    { "name": STORAGE_ACCOUNT_NAME }
)

if not availability_result.name_available:
    print(f"Storage name {STORAGE_ACCOUNT_NAME} is already in use. Try another name.")
    exit()

# The name is available, so provision the account
poller = storage_client.storage_accounts.begin_create(RESOURCE_GROUP_NAME, STORAGE_ACCOUNT_NAME,
    {
        "location" : LOCATION,
        "kind": "StorageV2",
        "sku": {"name": "Standard_LRS"}
    }
)

while poller.status() != 'Succeeded':
    pass
print(f'Creation of ' + STORAGE_ACCOUNT_NAME + ' ' + poller.status())

Creation of storage28344 Succeeded


# Create blob container 

In [9]:
# Step 3: Retrieve the account's primary access key and generate a connection string.
keys = storage_client.storage_accounts.list_keys(RESOURCE_GROUP_NAME, STORAGE_ACCOUNT_NAME)

print(f"Primary key for storage account: {keys.keys[0].value}")

conn_string = f"DefaultEndpointsProtocol=https;EndpointSuffix=core.windows.net;AccountName={STORAGE_ACCOUNT_NAME};AccountKey={keys.keys[0].value}"

print(f"Connection string: {conn_string}")

# Step 4: Provision the blob container in the account (this call is synchronous)
CONTAINER_NAME = "blob-container-01"
container = storage_client.blob_containers.create(RESOURCE_GROUP_NAME, STORAGE_ACCOUNT_NAME, CONTAINER_NAME, {})

# The fourth argument is a required BlobContainer object, but because we don't need any
# special values there, so we just pass empty JSON.

print(f"Provisioned blob container {container.name}")


Primary key for storage account: QQI3ZffwDIINtEsLFBdNbJPi1PwDCb7xyWqvM+1gjIe+KXJ1bG5Sm8/eu30dE3DctYzOG4wKJRkWvRGarAaTJQ==
Connection string: DefaultEndpointsProtocol=https;EndpointSuffix=core.windows.net;AccountName=storage28344;AccountKey=QQI3ZffwDIINtEsLFBdNbJPi1PwDCb7xyWqvM+1gjIe+KXJ1bG5Sm8/eu30dE3DctYzOG4wKJRkWvRGarAaTJQ==
Provisioned blob container blob-container-01


# Manage blobs

In [10]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

try:
    print("Azure Blob storage v" + __version__ + " - Python quickstart sample")
    # Quick start code goes here
except Exception as ex:
    print('Exception:')
    print(ex)

Azure Blob storage v12.6.0 - Python quickstart sample


In [11]:
connect_str = conn_string#os.getenv('AZURE_STORAGE_CONNECTION_STRING')

In [12]:
# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [25]:
# Create a unique name for the container
container_name = "quickstart" + str(uuid.uuid4())
container_name

'quickstart4baf93b2-7b66-497a-88ce-e77110a13660'

In [26]:

# Create the container
container_client = blob_service_client.create_container(container_name)
print(f"Provisioned blob container {container_client.container_name}")

Provisioned blob container quickstart4baf93b2-7b66-497a-88ce-e77110a13660


# Upload blobs to a container


In [31]:
# Create a file in local data directory to upload and download
local_path = "./data"
local_file_name = "quickstart" + str(uuid.uuid4()) + ".txt"
upload_file_path = os.path.join(local_path, local_file_name)

# Write text to the file
file = open(upload_file_path, 'w')
file.write("Hello, World!")
file.close()

# Create a blob client using the local file name as the name for the blob
blob_client = blob_service_client.get_blob_client(container=container_name, blob=local_file_name)

print("\nUploading to Azure Storage as blob:\n\t" + local_file_name)

# Upload the created file
with open(upload_file_path, "rb") as data:
    blob_client.upload_blob(data)



Uploading to Azure Storage as blob:
	quickstart5f9a5c54-efb0-4db1-9ede-c4ee440ca5f3.txt


# List the blobs in a container


In [32]:
print("\nListing blobs...")

# List the blobs in the container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print("\t" + blob.name)


Listing blobs...
	quickstart3d897e46-70b3-47a3-97a8-efc63e6615ae.txt
	quickstart5f9a5c54-efb0-4db1-9ede-c4ee440ca5f3.txt
	quickstartc2b71d99-eb55-4d91-98bb-7f78c810f8bb.txt


# Download blobs

In [34]:
# # Download the blob to a local file
# Add 'DOWNLOAD' before the .txt extension so you can see both files in the data directory
download_file_path = os.path.join(local_path, str.replace(local_file_name ,'.txt', 'DOWNLOAD.txt'))
print("\nDownloading blob to \n\t" + download_file_path)

with open(download_file_path, "wb") as download_file:
    download_file.write(blob_client.download_blob().readall())


	./data/quickstart5f9a5c54-efb0-4db1-9ede-c4ee440ca5f3DOWNLOAD.txt
